# FETCH ARTICLES ABSTRACTS FROM SCOPUS SEARCH

## Miguel Brito


First you need an API KEY from Scopus, get it from https://dev.elsevier.com

In [1]:

API_KEY="dad9618fb94619c7600007661fff829e"


Your query in here

In [2]:
QUERY="MACHINE LEARNING FORECAST DEMAND "

In [3]:
import requests
import json
import pandas as pd

def get_scopus_ids():
    resp = requests.get("http://api.elsevier.com/content/search/scopus?query=%s" % (QUERY),
                        headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    results= json.loads(resp.text.encode('utf-8'))
    return [[str(r['dc:identifier'])] for r in results['search-results']["entry"]]

In [5]:
#import textwrap
def get_scopus_info(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/abstract/scopus_id/"+ SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})

    return json.loads(resp.text.encode('utf-8'))

In [6]:
def get_scopus_text(SCOPUS_ID):
    url = ("https://api.elsevier.com/content/article/scopus_id/" + SCOPUS_ID + "?view=FULL")
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey': API_KEY})
    return json.loads(resp.text.encode('utf-8'))

In [24]:
scopus_ids=get_scopus_ids()
print("number of scopus ids fetched : %d " % len(scopus_ids))
article_set=''
article_set=pd.DataFrame()
n=0
for scopus_id in scopus_ids:
    results_info = get_scopus_info(scopus_id[0])
    results_text = get_scopus_text(scopus_id[0])
    title=results_info['abstracts-retrieval-response']['coredata']['dc:title']
    pubDate=results_info['abstracts-retrieval-response']['coredata']['prism:coverDate']
    try:
        abstract=results_text['full-text-retrieval-response']['coredata']['dc:description']
    except:
        abstract='(not available)'
        
    author=[[str(row['ce:indexed-name'])] for row in results_info['abstracts-retrieval-response']['coredata']['dc:creator']['author']]
    print("%d %s , %s , %s\n"  %( n, title,pubDate,author))
    #print("%s \n"  %(abstract))
    article_set[n,'Scopus_ID']=scopus_id[0]
    article_set[n,'title']=title
    article_set[n,'pubDate']=pubDate
    article_set[n,'abstract']=abstract
    article_set[n,'author']=author[0][0]
    n=n+1
    

number of articles fetched : 25 
0 Modeling and forecasting hotel room demand based on advance booking information , 2018-06-01 , [['Lee M.']]

1 A scaling law for short term load forecasting on varying levels of aggregation , 2018-06-01 , [['Sevlian R.']]

2 Workload prediction in cloud using artificial neural network and adaptive differential evolution , 2018-04-01 , [['Kumar J.']]

3 Effects of user-provided photos on hotel review helpfulness: An analytical approach with deep leaning , 2018-04-01 , [['Ma Y.']]

4 Short term wind power forecasting using hybrid variational mode decomposition and multi-kernel regularized pseudo inverse neural network , 2018-04-01 , [['Naik J.']]

5 Applying 3D texture algorithms on MRI to evaluate quality traits of loin , 2018-04-01 , [['Avila M.']]

6 PV based EV charging at universities using supplied historical PV output ramp , 2018-04-01 , [['Islam M.S.']]

7 Novel probabilistic optimization model for lead-acid and vanadium redox flow batteries und

In [22]:
article_set

,"(0, Scopus_ID)","(0, title)","(0, pubDate)","(0, abstract)","(0, author)","(1, Scopus_ID)","(1, title)","(1, pubDate)","(1, abstract)","(1, author)"
